In [16]:
import os
import shutil
from matplotlib import pyplot
import matplotlib.pyplot as plt
from matplotlib.image import imread

In [17]:
dataset_dir = 'dogvscat/'
base_dir = 'dogvscat_small'
os.makedirs(base_dir, exist_ok=True)

# Directory structure
for subset in ['train', 'validation', 'test']:
    for category in ['dogs', 'cats']:
        dir_path = os.path.join(base_dir, subset, category)
        os.makedirs(dir_path, exist_ok=True)